<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#YAHOO電影爬蟲練習" data-toc-modified-id="YAHOO電影爬蟲練習-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>YAHOO電影爬蟲練習</a></span><ul class="toc-item"><li><span><a href="#這個作業主要是練習爬取YAHOO電影的資訊，主要會用到基本的GET-method，以及解析伺服器回傳的html，從中擷取我們想要的資料。" data-toc-modified-id="這個作業主要是練習爬取YAHOO電影的資訊，主要會用到基本的GET-method，以及解析伺服器回傳的html，從中擷取我們想要的資料。-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>這個作業主要是練習爬取YAHOO電影的資訊，主要會用到基本的GET method，以及解析伺服器回傳的html，從中擷取我們想要的資料。</a></span><ul class="toc-item"><li><span><a href="#這個範例示範如何爬取上映中的電影資訊。先訪問YAHOO電影首頁：https://movies.yahoo.com.tw-，點選「上映中」的分頁，先確認上映中的電影總共有幾筆，然後逐頁爬取。" data-toc-modified-id="這個範例示範如何爬取上映中的電影資訊。先訪問YAHOO電影首頁：https://movies.yahoo.com.tw-，點選「上映中」的分頁，先確認上映中的電影總共有幾筆，然後逐頁爬取。-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>這個範例示範如何爬取上映中的電影資訊。先訪問YAHOO電影首頁：<a href="https://movies.yahoo.com.tw" target="_blank">https://movies.yahoo.com.tw</a> ，點選「上映中」的分頁，先確認上映中的電影總共有幾筆，然後逐頁爬取。</a></span></li><li><span><a href="#爬取所有上映中的電影資料：電影名稱、期待度、滿意度、簡介等等。" data-toc-modified-id="爬取所有上映中的電影資料：電影名稱、期待度、滿意度、簡介等等。-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>爬取所有上映中的電影資料：電影名稱、期待度、滿意度、簡介等等。</a></span></li></ul></li></ul></li></ul></div>

# YAHOO電影爬蟲練習
## 這個作業主要是練習爬取YAHOO電影的資訊，主要會用到基本的GET method，以及解析伺服器回傳的html，從中擷取我們想要的資料。

### 這個範例示範如何爬取上映中的電影資訊。先訪問YAHOO電影首頁：https://movies.yahoo.com.tw ，點選「上映中」的分頁，先確認上映中的電影總共有幾筆，然後逐頁爬取。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [3]:
# 先觀察一下目前上映中的電影數量
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("div", attrs={'class':'release_box'})  # 尋找正在上映中的全部電影筆數，其所在的tag
print("正在上映中總共: ", html.p.string)

正在上映中總共:  共84筆，目前顯示1~10筆


### 爬取所有上映中的電影資料：電影名稱、期待度、滿意度、簡介等等。

In [38]:
# 接下來要爬取每一頁的電影介紹
# 先點選網頁最底部的下一頁，觀察其網址變化。然後我們把頁數當作變數傳入，用迴圈爬取。
max_page = 5
for page_number in range(1, max_page+1, 1):
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
    payload = {'page':str(page_number)}
    resp = requests.get(url, params=payload)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')

    # 把電影介紹擷取出來
    movie_list = soup.find("ul", attrs={"class":"release_list"}).find_all("li")
    for p in movie_list:
        print("----------------------------------------------------------------------")
        # 電影名稱
        movie_name = p.find("div", attrs={"class":"release_movie_name"})
        print("電影名稱：", movie_name.a.string)
        
        # 定位電影評價資訊欄位
        level_box = movie_name.find("dl", attrs={"class":"levelbox"})
#         print(level_box)
        # 期待度
        expectation = level_box.find("div", attrs={"class":"leveltext"})
        print("期待度：", expectation.span.string)
        
        # 滿意度
#         satisfaction = level_box.find("div", attrs={"class":"leveltext starwithnum"})
        # 先用 find_all 找出所有 div tag 而且它的 class 為 level text，再用 [1] 取第二個元素
        satisfaction_list = level_box.find_all("div", attrs={"class":"leveltext"})[1]
        # 再取 span tag 該 class 為 count ，而其中一個 attribute 為 data-num，使用 ["data-num"] 取出該資料
        satisfaction = satisfaction_list.find("span", {"class": "count"})["data-num"]
#         print(satisfaction)
        print("滿意度：", satisfaction)  # 滿意度可以從"data-num"這個屬性擷取
        
        # 簡介
        movie_info = p.find("div", attrs={"class":"release_text"})
        print(movie_info.span.string)


----------------------------------------------------------------------
電影名稱： 
                  求婚好意外
期待度： 88%
滿意度： 4.2

                  艾比(克莉絲汀史都華 飾演)原本打算趁聖誕假期在家族聚餐上向女友哈珀(麥坎西戴維斯 飾演)求婚，沒想到哈珀壓根還沒跟保守的家人出櫃。關係曝光讓原本歡樂的假期陷入一團混亂，這對戀人該如何面對自己和親友勇敢說愛？同時又不毀掉完美的聖誕假期？
                
----------------------------------------------------------------------
電影名稱： 
                  怵目驚魂28天
期待度： 64%
滿意度： 4.3

                  開啟燒腦話題，全球推崇獨立經典，傑克葛倫霍影史最具代表性作品。
 
資質聰穎的唐尼想法異於同儕，一直以來都被解讀為怪胎，被學校認為是問題學生，被家人認為精神狀態異常，因此之前闖下的禍而必須接受心理醫生諮詢。
 
一天唐尼接受到奇幻人物的訊息，警告他世界末日即將在28天後到來。當天晚上，一架飛機的引擎掉落進東尼家的屋頂，唐尼因為奇幻人物的召喚逃過一劫。
 
看似平靜的校園裡，人與人之間充滿不了解與衝突，隱藏著未知的危機。隨著倒數的時間逐漸接近，唐尼陸續做出毀滅性的脫軌行為。末日來臨前的最後一刻，唐尼突然驚覺，他是唯一可以扭轉世界命運的人。
                
----------------------------------------------------------------------
電影名稱： 
                  信用詐欺師JP：公主篇
期待度： 95%
滿意度： 4.4

                  ★峰迴路轉燒腦歡快，日本賣座神片續集，原班人馬重磅出擊！
★ 蟬聯五週日本票房排行榜！超越前作大賣40億日幣!
★「眼睛看到的未必是真實。什麼是真，什麼是假。你能看穿真實嗎！？歡迎來到詐欺師的世界。」
★電影主題曲-Laughter【Official 鬚男dism】深情獻唱
 
《公主篇》以馬來西亞

----------------------------------------------------------------------
電影名稱： 
                  女王特大號
期待度： 98%
滿意度： 4.3

                  ★ 《愛情限時簽》導演安弗萊契最新作品
★ 改編自茱莉墨菲(Julie Murphy)同名暢銷小說
★ 金球獎得主珍妮佛安妮斯頓自製自演新作
★ 入圍第77屆金球獎電影類最佳原創歌曲
★ 入圍第62屆葛萊美獎佳影視媒體作品歌曲
★ 美麗，不一定要瘦！
 
薇樂蒂（丹妮爾麥當勞 飾）從小活在名人媽媽蘿絲（珍妮佛安妮斯頓 飾）的光環底下，由於天生長得胖嘟嘟，令她經常成為被訕笑的對象，甚至連媽媽都幫她取了個綽號「大飽」。另一方面，選美皇后出身的蘿絲，對女兒的身材也頗有意見，這也讓薇樂蒂經常質疑自己，難道吃飽飽就沒人愛嗎？某日，薇樂蒂在替陪伴她成長的露西阿姨整理遺物時，竟發現她年輕時曾想參加選美比賽，卻因自己的身材肥胖而打了退堂鼓。薇樂蒂為了完成阿姨的心願及不被媽媽看貶，她決定鼓起勇氣參加選美比賽，挑戰傳統審美觀並向世人宣示：「我胖我驕傲，有自信的人最美」！
 
                
----------------------------------------------------------------------
電影名稱： 
                  搖滾變色龍：大衛鮑伊
期待度： 100%
滿意度： 0.0

                  與成名之前的大衛鮑伊相遇吧！1971年， 24歲的大衛鮑伊展開了首次美國之旅，即將與尚未準備好迎接他的世界相遇。 
 
《搖滾變色龍》以大衛鮑伊的創作靈感和生命歷程為主軸，描繪出他最具代表性的舞台人格 Ziggy Stardust及這位世界級文化偶像的蛻變轉型。
 
【關於電影】
音樂史上最重要的偶像之一，那位隱藏在許多舞台角色背後的年輕人到底是誰？1971年， 24歲的大衛鮑伊（強尼佛林飾演）和水星唱片的發行公關羅伯·奧伯曼 (馬克馬龍飾演)一同展開了首次美國之旅，即將與尚未準備好迎接他的世界相遇。Ziggy Stardust是大衛鮑伊第一個也是最令人難忘的舞台人格，是讓他能成為全球最重要的文化偶像之一的轉捩點。
 
《搖

IndexError: list index out of range

### 關鍵解析

In [ ]:
#上面第28-31行

# 先用 find_all 找出所有 div tag 而且它的 class 為 level text，再用 [1] 取第二個元素
satisfaction_list = level_box.find_all("div", attrs={"class":"leveltext"})[1]
# 再取 span tag 該 class 為 count ，而其中一個 attribute 為 data-num，使用 ["data-num"] 取出該資料
satisfaction = satisfaction_list.find("span", {"class": "count"})["data-num"]


### HTML 長相

![](https://i.imgur.com/10rmG1J.png)
